# Advent of Code Day 20

In [ ]:
from utils import read_input

In [ ]:
def read_particles(filename):
    
    lines = read_input(filename)
    
    particles = [parse_particle_input(line) for line in lines]
    
    return particles

def parse_particle_input(particle_element):
    parts = [p.strip() for p in particle_element.split('>,')]
  
    position = tuple([int(p) for p in parts[0][3:].split(',')])
  
    velocity =  tuple([int(p) for p in parts[1][3:].split(',')])
 
    acceleration = tuple([int(p) for p in parts[2][3:-1].split(',')])
    
    return (position, velocity, acceleration)

### Part One Functions

For part one, we really only need 2 functions - one to compute the Manhattan distance from two 3D vectors (as tuples).  The other is to produce a new vector (as tuple) given two vectors. 

In [ ]:
def manhattan_distance(one, two):
    
    return abs(one[0] - two[0]) + abs(one[1] - two[1]) + abs(one[2] - two[2])

def add_vectors(one, two):
    return (one[0] + two[0], one[1] + two[1], one[2] + two[2])

### Part One

The key to this problem is the acceleration of each particle will never change.  Every particle begins with some position and velocity but both are adjusted by their acceleration each time unit.  It may be the case that some particles will move temporarily towards the origin, but long-term, all will be moving away from the origin.  The question is: which one will be closest?  The one whose acceleration is the slowest.  To determine that, we'll just use Manhattan distance of the acceleration for each particle and the minimum such acceleration is for the particle that will ultimately be closest.  Despite what the problem implies, we never have to simulate anything.

In [ ]:
def solve_part_one():
       
    origin = (0, 0, 0)

    p = read_particles('Input/day20.txt')
    
    accelerations = [acc for (_, _, acc) in p]

    distances = [manhattan_distance(acc, origin) for acc in accelerations]
    
    print 'Particle {} will always be closest to origin'.format(np.argmin(distances))
    

In [ ]:
solve_part_one()

### Part Two

There are a few options to find colliding particles but I'll use a rather bruteforce one and simulate movements of the particles and look for collisions.  There are physics equations to compute position for any unit of time given starting velocity and acceleration, but we'll just make stepwise updates to velocity and position.  First, update_particles takes a collection of tuples (position, velocity, acceleration) and uses update_particle to produce a new collection that has the same acceleration but updated velocity and position.  Then who_survives returns only those particles that survive based on position, using survives to determine if any other particle shares the same position for each particle.  The only real question is: how do we know when all collisions have occurred?  We'll just take a gamble that 1000 time units is sufficient for all collisions to have occurred and that no particles end up on the same path with one trailing the other.  It's possible to determine a stopping point based on having all particles moving away from the origin and none sharing a normalized vector, but we'll just go the easy route.

In [ ]:
def update_particle(particle):
    
    ((x, y, z), (vx, vy, vz), (ax, ay, az)) = particle
    
    acc = (ax, ay, az)
    vel = (vx + ax, vy + ay, vz + az)
    pos = (x + vel[0], y + vel[1], z + vel[2])
    
    return (pos, vel, acc)

def update_particles(particles):
    
    return [update_particle(particle) for particle in particles]


def survives(current_index, particles):
    
    for ix, (pos, vel, acc) in enumerate(particles):
        if particles[current_index][0] == pos and ix != current_index:
            return False
        
    return True

def who_survives(particles):
    
    return [particles[ix] for ix in xrange(0, len(particles)) if survives(ix, particles)]

In [ ]:
def solve_part_two():
        
    particles = read_particles('Input/day20.txt')
    
    for iteration in xrange(1, 1000):
        
        particles = update_particles(particles)
        
        particles = who_survives(particles)
    
    print '{} particles survived'.format(len(particles))
 

In [ ]:
solve_part_two()